In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

from aux import K, Cl, clxc, Windows
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap

plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')
plt.rc('font', size=15)
plt.rc('figure', autolayout=True)
plt.rc('axes', titlesize=16, labelsize=17)
plt.rc('lines', linewidth=2, markersize=6)
plt.rc('legend', fontsize=15)
plt.rc('figure')

# Do not forget to compile the Fortran integrator! 
# > f2py -c CrossCorr_Cells.f95 -m CrossCorr_Cells

In [ ]:
# Parameters for the fiducial model/window functions
bGW = 1.
K0_bsqr = 1.
z_pivot = 1.
Om = 0.32
larray = np.arange(10,500)


k_max_lst = np.array([0.32, 0.30, 0.28, 0.26, 0.24, 0.22, 0.20, 0.18, 0.16, 0.14, 0.12, 0.10])

In [ ]:
plt.style.use('seaborn-dark-palette')
fig = plt.figure()


colorparams = k_max_lst[::-1]
normalize = mcolors.Normalize(vmin=np.min(colorparams), vmax=np.max(colorparams))
colormap = ListedColormap(sns.cubehelix_palette(12, reverse = False).as_hex())#cm.viridis


for indx, k_max_tmp in enumerate(k_max_lst):
    
    clxcarray = np.load('../Data/grid/F/k_range_tests/' + str(indx + 1) + '.npy')
    
    a = np.array(clxcarray[0])
    z = 1./a - 1.
    K_kernel_Cl, K_kerneln = K(z, K0_bsqr, z_pivot, 0.32, bGW)
    
    spectrum = 1 #1 == auto, 2 == cross
    Clarray = Cl(larray, K_kernel_Cl, K_kernel_Cl, clxcarray, bGW, spectrum, Fortran=True)
    
    color = colormap(normalize(k_max_tmp))
    plt.plot(larray, Clarray*larray*(larray + 1), color=color)

# Colorbar setup
s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
s_map.set_array(colorparams)

halfdist = (colorparams[1] - colorparams[0])/2.0
boundaries = np.linspace(colorparams[0] - halfdist, colorparams[-1] + halfdist, len(colorparams) + 1)

cbar = fig.colorbar(s_map, spacing='proportional', ticks=colorparams, boundaries=boundaries, format='%2.2g')

cbarlabel = r'$k_\mathrm{max}[\mathrm{Mpc^{-1}}]$'
cbar.set_label(cbarlabel, fontsize=20)
        


import seaborn as sns
        
plt.gca().set_xlim([10, 99])
plt.gca().set_ylim([4e4, 21e4])
#plt.gca().set_yscale("log")
    
plt.gca().set_xlabel(r"$\ell$")
plt.gca().set_ylabel(r"$\ell(\ell + 1)C_\ell^{\mathrm{GW}}$")
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,1))
    
plt.gca().grid(True, which = 'major', alpha = 0.3, ls = "-")

plt.savefig("Figs/autocorrel_k_depend_shorter_range.pdf")


In [ ]:
from aux import K, Cl, clxc, Windows
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import seaborn as sns

plt.style.use('seaborn-dark-palette')
fig = plt.figure()


larray = np.arange(10, 500)
d = np.sqrt(2./(2.*larray + 1.))

colorparams = k_max_lst[::-1]
normalize = mcolors.Normalize(vmin=np.min(colorparams), vmax=np.max(colorparams))
colormap = ListedColormap(sns.cubehelix_palette(12, reverse = False).as_hex())#cm.viridis



for indx, k_max_tmp in enumerate(k_max_lst):
    
    clxcarray = np.load('../Data/grid/F/k_range_tests/' + str(indx + 1) + '.npy')
    
    a = np.array(clxcarray[0])
    z = 1./a - 1.
    W_kernel = norm.pdf(z, loc=0.5, scale=0.1)
    
    K_kernel_Cl, K_kerneln = K(z, K0_bsqr, z_pivot, 0.32, bGW)
    
    spectrum = 2 #1 == auto, 2 == cross
    Clarray = Cl(larray, K_kernel_Cl, W_kernel, clxcarray, bGW, spectrum, Fortran=True)
    
    color = colormap(normalize(k_max_tmp))
    plt.plot(larray, Clarray*larray*(larray + 1), color=color)

# Colorbar setup
s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
s_map.set_array(colorparams)

halfdist = (colorparams[1] - colorparams[0])/2.0
boundaries = np.linspace(colorparams[0] - halfdist, colorparams[-1] + halfdist, len(colorparams) + 1)

cbar = fig.colorbar(s_map, spacing='proportional', ticks=colorparams, boundaries=boundaries, format='%2.2g') # format='%2i' for integer


cbarlabel = r'$k_\mathrm{max}[\mathrm{Mpc^{-1}}]$'
cbar.set_label(cbarlabel, fontsize=20)
        

plt.style.use('seaborn-dark-palette')

        
plt.gca().set_xlim([10, 99])
plt.gca().set_ylim([0., 60.])
#plt.gca().set_yscale("log")
    
plt.gca().set_xlabel(r"$\ell$")
plt.gca().set_ylabel(r"$\ell(\ell + 1)C_\ell^{\mathrm{GW}\times\mathrm{GC}}$")
    
plt.gca().grid(True, which = 'major', alpha = 0.3, ls = "-")

plt.savefig("Figs/crosscorrel_k_depend.pdf")